# simran4@wisc.edu, rgundavarapu@wisc.edu

# Part 3: Spark Streaming

In [1]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder.appName("cs544")
         .config("spark.sql.shuffle.partitions", 10)
         .config("spark.ui.showConsoleProgress", False)
         .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.2')
         .getOrCreate())

df = (
    spark.readStream.format("kafka")
    .option("kafka.bootstrap.servers", "kafka:9092")
    .option("subscribe", "stations-json")
    .option("startingOffsets", "earliest")
    .load()
)

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-22eb1df4-4d36-41a6-9a9f-1267d7eeb077;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.2 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.ap

Use a spark.readStream to load the stations-json stream to a DataFrame. Tips:

the value comments will contain the bytes you wrote. You can use col(????).cast("string") to convert bytes to a string (this assumes UTF-8 encoding)
you can convert a JSON string to a structure using from_json(????, schema). The schema needs to specify the types. It will be something like schema = "station STRING, date DATE, ..." for you.
if a column named "value" is a struct, you can access an entry named "station" inside with "value.station"
Requirements

use .option("startingOffsets","earliest") to begin with the earliest data

In [2]:
from pyspark.sql.functions import col, from_json

In [3]:
from pyspark.sql import functions
schema = "station STRING, date DATE, degrees FLOAT, raining INTEGER"
data = (df.select(col("key").cast("string"),
          from_json(col("value").cast("string"), schema).alias("value")))

counts_df = data.groupBy('value.station').agg(functions.min('value.date').alias('start'),
                                              functions.max('value.date').alias('end'),
                                              functions.count('value.station').alias('measurements'),
                                              functions.avg('value.degrees').alias('avg'),
                                              functions. max('value.degrees').alias('max')).sort('value.station')
                 
s = counts_df.writeStream.format("console").trigger(processingTime="5 seconds").outputMode("complete").start()
s.awaitTermination(30)
s.stop()

23/04/30 02:26:01 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-f49ec4f8-8c11-413f-ac99-da22e5a0bfb5. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/04/30 02:26:01 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-------+----------+----------+------------+------------------+----------+
|station|     start|       end|measurements|               avg|       max|
+-------+----------+----------+------------+------------------+----------+
|      A|2000-01-01|2001-05-22|         508| 65.90495668246051|107.278824|
|      B|2000-01-01|2001-05-22|         508| 43.17990932983207| 92.984245|
|      C|2000-01-01|2001-05-22|         508| 48.03097364083519| 98.619316|
|      D|2000-01-01|2001-05-22|         508|53.842927794756854| 103.26719|
|      E|2000-01-01|2001-05-22|         508| 61.21108461740449|111.860794|
|      F|2000-01-01|2001-05-22|         508|  53.8868232824671|110.423775|
|      G|2000-01-01|2001-05-22|         508| 53.72359361423282|100.010826|
|      H|2000-01-01|2001-05-22|         508| 66.30751238470002| 116.85257|
|      I|2000-01-01|2001-05-22|         508| 37.64841536461838|  81.53725|
|  

23/04/30 02:26:17 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 15658 milliseconds


-------------------------------------------
Batch: 1
-------------------------------------------
+-------+----------+----------+------------+------------------+----------+
|station|     start|       end|measurements|               avg|       max|
+-------+----------+----------+------------+------------------+----------+
|      A|2000-01-01|2001-06-02|         519| 66.15168322671585|107.278824|
|      B|2000-01-01|2001-06-02|         519|  43.4042908544715| 92.984245|
|      C|2000-01-01|2001-06-02|         519| 48.15941479107318| 98.619316|
|      D|2000-01-01|2001-06-02|         519| 54.17535851410588| 103.26719|
|      E|2000-01-01|2001-06-02|         519| 61.41431935260751|111.860794|
|      F|2000-01-01|2001-06-02|         519|54.161726527131364|110.423775|
|      G|2000-01-01|2001-06-02|         519| 54.19844453550717|100.010826|
|      H|2000-01-01|2001-06-02|         519| 66.78008594770192| 116.85257|
|      I|2000-01-01|2001-06-02|         519| 37.91120454387628|  81.53725|
|  

In [5]:
today=data.select(col('value.station'),col('value.date'),col('value.raining'))
today

DataFrame[station: string, date: date, raining: int]

In [6]:
from pyspark.sql.functions import month,date_add
features_yesterday=data.select(col('value.raining').alias('sub1raining'),col('value.degrees').alias('sub1degrees'),col('value.station'),month(col('value.date')).alias('month'),date_add('value.date', 1).alias('date'))
features_2days=data.select(col('value.raining').alias('sub2raining'),col('value.degrees').alias('sub2degrees'),col('value.station'),month(col('value.date')).alias('month'),date_add('value.date', 2).alias('date'))


In [7]:
features=features_2days.join(features_yesterday,['date','month','station'],"inner") 
#features_2days.station ==  features_yesterday.station,features_2days.date ==  features_yesterday.date
features

DataFrame[date: date, month: int, station: string, sub2raining: int, sub2degrees: float, sub1raining: int, sub1degrees: float]

In [8]:
final= today.join(features,['date','station'],'inner')
#today.date==features.date,today.station==features.station
final

DataFrame[date: date, station: string, raining: int, month: int, sub2raining: int, sub2degrees: float, sub1raining: int, sub1degrees: float]

In [17]:
s3 = final.repartition(1).writeStream.format("parquet").option("path", "./today_features").option("checkpointLocation","./checkpoint/directory").trigger(processingTime="1 minute").start()


23/04/30 02:44:57 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/04/30 02:45:00 WARN CheckpointFileManager: Failed to rename temp file file:/notebooks/checkpoint/directory/sources/0/.0.6e4aeac1-2474-49ca-a9a9-6e52d07c7873.tmp to file:/notebooks/checkpoint/directory/sources/0/0 because file exists
org.apache.hadoop.fs.FileAlreadyExistsException: Rename destination file:/notebooks/checkpoint/directory/sources/0/.0.crc already exists.
	at org.apache.hadoop.fs.AbstractFileSystem.renameInternal(AbstractFileSystem.java:770)
	at org.apache.hadoop.fs.AbstractFileSystem.rename(AbstractFileSystem.java:720)
	at org.apache.hadoop.fs.ChecksumFs.renameInternal(ChecksumFs.java:496)
	at org.apache.hadoop.fs.AbstractFileSystem.rename(AbstractFileSystem.java:720)
	at org.apache.hadoop.fs.FileContext.rename(FileContext.java:1036)
	at org.apache.spark.sql.execution.streaming.FileContextBasedCheckpointFileManager.renameTempFi

# Part 4: Spark ML

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
today

In [ ]:
features

In [ ]:
import pandas as pd
pd.read_parquet('today_features/part-00000-08f2e6dc-9c9a-4945-b8a3-bac5b9e6105d-c000.snappy.parquet')